In [ ]:
from os import listdir
from os.path import isfile, join


In [5]:
import netCDF4 as nc
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rcParams

In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-d

In [2]:
#if working on lx-moissard
Cluster = 'Occ/'
run_name = '20_08_18_new_big_one_0'
filepath = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/p3_files/'


In [3]:
posx = []
posy = []
posz = []

velx = []
vely = []
velz = []

i = 0

for p3_file in listdir(filepath):
    
    if not(".nc" in p3_file):
        continue
    
    i+=1
        
    print("Reading file", i, "/", len(listdir(filepath)))
    print("Reading ", p3_file, " ...")

    p3_data = nc.Dataset(filepath + p3_file,'r')
    
    posx = posx + list(np.array(p3_data.variables['particule_x']))
    posy = posy + list(np.array(p3_data.variables['particule_y']))
    posz = posz + list(np.array(p3_data.variables['particule_z']))

    velx = velx + list(-np.array(p3_data.variables['particule_vx']))
    vely = vely + list(-np.array(p3_data.variables['particule_vy']))
    velz = velz + list(np.array(p3_data.variables['particule_vz']))


FileNotFoundError: [Errno 2] No such file or directory: b'/data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/p3_files/p3_0000_18_08_20_t00230.nc'

In [1]:
# /!\ ugly hard coding

nx = 1500
ny = 720
nz = 660
dx = 1
dy = 1
dz = 1
px = (nx - 100)*dx
py = ny/2 * dy
pz = nz/2 * dz

def real_pos(pos_part, planet, sgn):
    return sgn*(np.array(pos_part) - planet*np.ones(len(pos_part)))

# /!\ Verify this, I think y might need s=-1. See read_netcdf_3D.py
rposx = real_pos(posx, px, -1)
rposy = real_pos(posy, py, -1)
rposz = real_pos(posz, pz,  1)

In [ ]:
#for '20_08_18_new_big_one_0', at t=230
# cube = (xmin, xmax, ymin, ymax, zmin, zmax)
cube_sw = (-68-20, -68-10, 180-5, 180+5, -5, 5)
cube_bs = (-64-20, -64-10, 135-5, 135+5, -5, 5)
cube_mp = (-41-20, -41-10, 76 -5, 76 +5, -5, 5)

In [ ]:
def incube(point, cube):
    x, y, z = point
    xmin, xmax, ymin, ymax, zmin, zmax = cube
    return (     (xmin < x < xmax) 
             and (ymin < y < ymax)
             and (zmin < z < zmax) )

In [2]:
print("Constructing list of particles in the solar wind")

x_sw = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
y_sw = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
z_sw = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]

vA = 89.371 #Ugly hard code
vx_sw = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
vy_sw = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
vz_sw = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]

NameError: name 'rposx' is not defined

In [ ]:
print("Constructing list of particles in the magnetosheath, near the bow shock")

x_bs = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
y_bs = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
z_bs = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]

vA = 89.371 #Ugly hard code
vx_bs = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
vy_bs = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
vz_bs = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]

In [ ]:
print("Constructing list of particles in the magnetosheath, near the magnetopause")

x_mp = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
y_mp = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
z_mp = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]

vA = 89.371 #Ugly hard code
vx_mp = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
vy_mp = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
vz_mp = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]

In [ ]:
def avg(data):
    return np.mean(data)

def rms(data):
    return np.mean(np.sqrt( (np.array(data) - np.array(avg(data)))**2))

In [ ]:
rcParams["figure.figsize"] = [5, 3.75]

plt.close('all')
plt.figure()
plt.scatter(vx_sw, vy_sw, s = 0.05, color='black')
# plt.title("Velocity distribution in the solar wind", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
plt.axis('equal')
# plt.xlim([avg(vx_sw)+8*rms(vx_sw), avg(vx_sw)-8*rms(vx_sw)])
# plt.ylim([avg(vy_sw)-8*rms(vy_sw), avg(vy_sw)+8*rms(vy_sw)])
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.figure()
plt.scatter(vx_bs, vy_bs, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the bow shock", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
# plt.xlim([avg(vx_bs)+8*rms(vx_bs), avg(vx_bs)-8*rms(vx_bs)])
# plt.ylim([avg(vy_bs)-8*rms(vy_bs), avg(vy_bs)+8*rms(vy_bs)])
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.figure()
plt.scatter(vx_mp, vy_mp, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the magnetopause", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.show()